In [1]:
# packages for data processing
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

# packages for linear model
import statsmodels
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
# convert date into datetime type
def parse_date(string):
    month, day, year = string.split('/')
    return datetime(int(year), int(month), int(day))

# extract number of days from datetime
def stay(time):
    return time.days

def remove_HUD(string):
    return string[:-6]

In [3]:
# read and select data with parameters of interest
client_background = ["Client ID", "Client Age at Entry", "Client Gender", "Client Primary Race", "Client Ethnicity", "Client Veteran Status"]
duration_interest = ['Client ID', 'Entry Date', 'Exit Date']
client = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-Jianqiao-Wang/master/project_3/data/CLIENT_191102.tsv", sep="\t")[client_background]
duration = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-Jianqiao-Wang/master/project_3/data/ENTRY_EXIT_191102.tsv", sep="\t")[duration_interest]

In [4]:
# join two datasets by client ID
data = pd.concat([duration, client], axis=1, join="inner")

# replace cells with NAN
data.replace('Data not collected (HUD)', np.nan, inplace=True)
data.replace("Client doesn't know (HUD)", np.nan, inplace=True)
data.replace('Client refused (HUD)', np.nan, inplace=True)

# remove missing data
data.dropna(inplace=True)

# calculate duration of people
data["Entry Date"] = data["Entry Date"].apply(parse_date)
data["Exit Date"] = data["Exit Date"].apply(parse_date)
data["duration"] = (data["Exit Date"] - data["Entry Date"]).apply(stay)
data = data.loc[data['duration']>0]

# rename some column names for later analysis
data.rename(columns={"Client Primary Race": "ClientPrimaryRace", 
                     "Client Ethnicity": "ClientEthnicity", 
                     "Client Veteran Status": "ClientVeteranStatus", 
                     "Client Age at Entry": "ClientAge", 
                     "Client Gender": "ClientGender"}, inplace=True)

# remove HUD tail
data["ClientPrimaryRace"]=data["ClientPrimaryRace"].apply(remove_HUD)
data["ClientEthnicity"]=data["ClientEthnicity"].apply(remove_HUD)
data["ClientVeteranStatus"]=data["ClientVeteranStatus"].apply(remove_HUD)

In [5]:
# save data
data.to_csv("/Users/jianqiaowang/Documents/GitHub/bios611-projects-fall-2019-Jianqiao-Wang/project_3/scripts/duration.csv")

In [6]:
# linear model to test effect of different covariates
model = smf.ols(formula='duration~ClientAge+ClientGender+ClientEthnicity+ClientPrimaryRace+ClientVeteranStatus', 
                data=data,).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     9.191
Date:                Sun, 17 Nov 2019   Prob (F-statistic):           6.14e-14
Time:                        20:33:48   Log-Likelihood:                -28474.
No. Observations:                5125   AIC:                         5.697e+04
Df Residuals:                    5115   BIC:                         5.703e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================================================================
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         11.2572      9.075      1.240      0.215      -6.534      29.048
ClientGender[T.Male]                                              -6.1826      2.086     -2.963      0.003     -10.273      -2.092
ClientGender[T.Trans Female (MTF or Male to Female)]              -6.6933     15.323     -0.437      0.662     -36.732      23.346
ClientEthnicity[T.Non-Hispanic/Non-Latino]                        10.9276      4.985      2.192      0.028       1.155      20.700
ClientPrimaryRace[T.Asian]                                        24.5839     36.844      0.667      0.505     -47.646      96.814
ClientPrimaryRace[T.Black or African American]                    -1.3704      6.918     -0.198      0.843     -14.932      12.192
ClientPrimaryRace[T.Native Hawaiian or Other Pacific Islander]    -6.2554     20.996     -0.298      0.766     -47.417      34.906
ClientPrimaryRace[T.White]                                        -4.3062      7.078     -0.608      0.543     -18.183       9.570
ClientVeteranStatus[T.Yes]                                        -6.9446      2.958     -2.347      0.019     -12.745      -1.145
ClientAge                                                          0.5775      0.071      8.142      0.000       0.438       0.717
==============================================================================
Omnibus:                     3839.613   Durbin-Watson:                   1.637
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            81059.215
Skew:                           3.428   Prob(JB):                         0.00
Kurtosis:                      21.237   Cond. No.                     1.94e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.94e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [7]:
smf.ols(formula='duration~ClientGender', 
                data=data,).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               duration   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     3.747
Date:                Sun, 17 Nov 2019   Prob (F-statistic):             0.0236
Time:                        20:33:55   Log-Likelihood:                -28512.
No. Observations:                5125   AIC:                         5.703e+04
Df Residuals:                    5122   BIC:                         5.705e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================================================
                                                           coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------
Intercept                                               44.0338      1.791     24.586      0.000      40.523      47.545
ClientGender[T.Male]                                    -5.5296      2.058     -2.686      0.007      -9.565      -1.494
ClientGender[T.Trans Female (MTF or Male to Female)]   -12.2691     15.407     -0.796      0.426     -42.473      17.935
==============================================================================
Omnibus:                     3879.195   Durbin-Watson:                   1.630
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            82989.027
Skew:                           3.477   Prob(JB):                         0.00
Kurtosis:                      21.447   Cond. No.                         22.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
data2 = pd.read_csv("https://raw.githubusercontent.com/datasci611/bios611-projects-fall-2019-Jianqiao-Wang/master/project_3/data/INCOME_EXIT_191102.tsv", sep="\t")

In [10]:
data2.dropna(inplace=True)

In [13]:
data2.head(5)

,EE Provider ID,EE UID,Client Unique ID,Client ID,ReceivingIncome (Exit),Source of Income (Exit),Monthly Amount (Exit),Income Start Date (Exit),Income End Date (Exit),Recordset ID (140-recordset_id),Provider (140-provider),Date Added (140-date_added)
162,Urban Ministries of Durham - Durham County - S...,687909,ctnm05251964c642t534,165730,Yes,SSI (HUD),733.0,2/28/1979,12/31/2017,1715660,Urban Ministries of Durham - Durham County(1562),3/2/2015
166,Urban Ministries of Durham - Durham County - S...,687911,rcam05241959r200c620,291392,No,Earned Income (HUD),800.0,8/1/2015,12/4/2018,3431695,Urban Ministries of Durham - Durham County - S...,2/9/2016
224,Urban Ministries of Durham - Durham County - S...,687914,mgam05151964m240g600,360237,Yes,Earned Income (HUD),380.0,3/8/2015,11/25/2015,1900332,Urban Ministries of Durham - Durham County(1562),4/8/2015
227,Urban Ministries of Durham - Durham County - S...,687951,kmlm12231983k150m420,403745,No,Earned Income (HUD),1200.0,8/12/2015,9/15/2015,2442837,Urban Ministries of Durham - Durham County - S...,8/19/2015
356,Urban Ministries of Durham - Durham County - S...,687982,cohf02011960c530o355,395548,Yes,SSDI (HUD),847.0,6/1/2006,3/21/2019,1821447,Urban Ministries of Durham - Durham County(1562),3/23/2015


,Client ID,Entry Date,Exit Date,Client ID,ClientAge,ClientGender,ClientPrimaryRace,ClientEthnicity,ClientVeteranStatus,duration
0,397941,2015-08-15,2016-07-11,397941,60.0,Female,White,Non-Hispanic/Non-Latino,No,331
1,130335,2015-08-15,2015-08-31,130335,48.0,Female,Black or African American,Non-Hispanic/Non-Latino,No,16
2,188933,2015-08-15,2015-09-19,188933,42.0,Female,Black or African American,Non-Hispanic/Non-Latino,No,35
3,168290,2015-08-15,2016-03-07,168290,57.0,Male,White,Hispanic/Latino,No,205
4,123122,2015-08-15,2015-08-24,123122,51.0,Male,White,Non-Hispanic/Non-Latino,No,9
5,363382,2015-08-15,2015-10-29,363382,30.0,Male,Black or African American,Non-Hispanic/Non-Latino,No,75
6,165730,2015-08-15,2015-09-06,165730,51.0,Male,Black or African American,Non-Hispanic/Non-Latino,No,22
7,291392,2015-08-15,2015-08-22,291392,56.0,Male,Black or African American,Non-Hispanic/Non-Latino,Yes,7
8,359808,2015-08-15,2015-10-29,359808,60.0,Male,Black or African American,Non-Hispanic/Non-Latino,Yes,75
9,385539,2015-08-15,2015-09-23,385539,59.0,Male,Black or African American,Non-Hispanic/Non-Latino,Yes,39
